In [15]:
import pandas as pd
import os
import psutil

# Step 1: Define Data Path
data_path = "D:/mimic-iv-3.1"

# Step 2: Check memory usage before loading data
def check_memory():
    print(f"Memory Usage: {psutil.virtual_memory().percent}%")
check_memory()


Memory Usage: 33.4%


In [5]:
# Load MIMIC-IV Tables with optimized memory handling
# =============================
# 1. Load and Process Admissions & Patients (One row per admission)
# =============================
admissions = pd.read_csv(
    os.path.join(data_path, "hosp/admissions.csv.gz"),
    usecols=["subject_id", "hadm_id", "admission_type", "admission_location", "discharge_location", 
             "admittime", "dischtime", "insurance", "race", "marital_status", "edregtime", "edouttime"],
    low_memory=False
)

# Calculate ED time spent (in minutes) and hospital length of stay (in days)
admissions["ed_time_spent"] = (pd.to_datetime(admissions["edouttime"]) - pd.to_datetime(admissions["edregtime"])).dt.total_seconds() / 60
admissions["los_hosp"] = (pd.to_datetime(admissions["dischtime"]) - pd.to_datetime(admissions["admittime"])).dt.total_seconds() / (60*60*24)

# Drop raw timestamp columns to reduce redundancy
admissions = admissions.drop(columns=["edregtime", "edouttime", "admittime", "dischtime"], errors="ignore")
 # Load patient demographics
patients = pd.read_csv(
    os.path.join(data_path, "hosp/patients.csv.gz"),
    usecols=["subject_id", "gender", "anchor_age", "anchor_year"],
    low_memory=False
)

In [5]:
print(admissions.head())  # Show first 5 rows

   subject_id   hadm_id  admission_type      admission_location  \
0    10000032  22595853          URGENT  TRANSFER FROM HOSPITAL   
1    10000032  22841357        EW EMER.          EMERGENCY ROOM   
2    10000032  25742920        EW EMER.          EMERGENCY ROOM   
3    10000032  29079034        EW EMER.          EMERGENCY ROOM   
4    10000068  25022803  EU OBSERVATION          EMERGENCY ROOM   

  discharge_location insurance marital_status   race  ed_time_spent  los_hosp  
0               HOME  Medicaid        WIDOWED  WHITE          253.0  0.786111  
1               HOME  Medicaid        WIDOWED  WHITE          337.0  1.015278  
2            HOSPICE  Medicaid        WIDOWED  WHITE          286.0  1.754167  
3               HOME  Medicaid        WIDOWED  WHITE          486.0  2.222222  
4                NaN       NaN         SINGLE  WHITE          511.0  0.298611  


In [9]:
# =============================
# 2A.  Diagnoses Data ICD Codes conversion
# =============================
# Load GEMs ICD-9 to ICD-10 Mapping File
gem_file_path = "D:/MIMIC-IV-Data-Pipeline/icd9toicd10cmgem.csv"  # Replace with the correct path
col_names = ["icd9_code", "icd10_code", "flags", "approximate", "no_map", "combination", "scenario", "choice_list"]

# Read the file (it's space-separated)
icd9_to_icd10_gem = pd.read_csv(gem_file_path, sep=",", header=0, names=col_names, dtype=str, skiprows=1)

# Display first few rows
print(icd9_to_icd10_gem.head())


  icd9_code icd10_code  flags approximate no_map combination scenario  \
0      0011       A001  00000           0      0           0        0   
1      0019       A009  00000           0      0           0        0   
2      0020      A0100  10000           1      0           0        0   
3      0021       A011  00000           0      0           0        0   
4      0022       A012  00000           0      0           0        0   

  choice_list  
0           0  
1           0  
2           0  
3           0  
4           0  


In [53]:
# =============================
# 2B. Load Diagnoses Data
# =============================
# Load your MIMIC-IV diagnoses data
 
diagnoses = pd.read_csv(
    os.path.join(data_path, "hosp/diagnoses_icd.csv.gz"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
    low_memory=False
)


#=============================
# 2A.  Diagnoses Data ICD Codes conversion
# =============================
# Separate ICD-9 diagnoses
icd9_diagnoses = diagnoses[diagnoses["icd_version"] == 9].copy()

# Standardize ICD-9 codes (strip spaces & remove decimal points)
icd9_diagnoses["icd_code"] = icd9_diagnoses["icd_code"].str.strip().str.split(".").str[0]

# Merge with the pre-mapped CSV file
mapped_icd10 = icd9_diagnoses.merge(icd9_to_icd10_gem, left_on="icd_code", right_on="icd9_code", how="left")

# Use ICD-10 codes where available, otherwise keep original ICD-9 codes
mapped_icd10["final_icd_code"] = mapped_icd10["icd10_code"].fillna(mapped_icd10["icd9_code"])

# Keep only relevant columns
mapped_icd10 = mapped_icd10[["subject_id", "hadm_id", "final_icd_code"]]

# Rename ICD-10 column for consistency
icd10_diagnoses = diagnoses[diagnoses["icd_version"] == 10].copy()
icd10_diagnoses = icd10_diagnoses.rename(columns={"icd_code": "final_icd_code"})
icd10_diagnoses = icd10_diagnoses[["subject_id", "hadm_id", "final_icd_code"]]

# Combine mapped ICD-10 diagnoses with original ICD-10 data
final_diagnoses = pd.concat([icd10_diagnoses, mapped_icd10], ignore_index=True)


In [54]:
print(final_diagnoses.head())  # Show the first few rows


   subject_id   hadm_id final_icd_code
0    10000084  23052089          G3183
1    10000084  23052089          F0280
2    10000084  23052089           R441
3    10000084  23052089           R296
4    10000084  23052089           E785


In [65]:
# =====================================
# 2B. Retain the first recorded (primary) diagnosis per admission
# ======================================

# Retain the first recorded (primary) diagnosis per admission
primary_diagnosis = final_diagnoses.groupby(["subject_id", "hadm_id"]).agg(
    primary_diagnosis=('final_icd_code', 'first')  # Take the first recorded diagnosis
).reset_index()


# =====================================
# 2C. Aggregate Diagnoses Per Admission
# ======================================
# Aggregate diagnoses per admission: count unique diagnoses and collect all codes as a list
diagnosis_counts = final_diagnoses.groupby(["subject_id", "hadm_id"]).agg(
    num_comorbidities=('final_icd_code', 'nunique'),
    diagnosis_list=('final_icd_code', lambda x: list(x))
).reset_index()


# Ensure no duplicate `diagnosis_list`
diagnosis_list_per_admission = diagnosis_counts[["subject_id", "hadm_id", "diagnosis_list"]].copy()

# Merge aggregated data while keeping primary diagnosis
diagnosis_counts = diagnosis_counts.drop(columns=["diagnosis_list"]).merge(
    diagnosis_list_per_admission, on=["subject_id", "hadm_id"], how="left"
)

# Now Merge Primary Diagnosis  
diagnosis_counts = diagnosis_counts.merge(primary_diagnosis, on=["subject_id", "hadm_id"], how="left")


In [66]:
print(diagnosis_counts.head())  # Show the first few rows

   subject_id   hadm_id  num_comorbidities  \
0    10000032  22595853                 11   
1    10000032  22841357                 10   
2    10000032  25742920                 14   
3    10000032  29079034                 16   
4    10000068  25022803                  1   

                                      diagnosis_list primary_diagnosis  
0  [K766, R188, K740, K7460, K7469, B1920, J449, ...              K766  
1  [B1921, R188, D696, E871, J449, K740, K7460, K...             B1921  
2  [B182, R188, Z9981, K740, K7460, K7469, E875, ...              B182  
3  [I952, I9581, B182, R64, E871, R188, E875, F17...              I952  
4                                            [F1010]             F1010  


In [95]:
# =============================
# 2D. Create Key Clinical Flags
# =============================
# Create key clinical flags based on diagnosis_list
# (These lambda functions operate on the aggregated list for each admission.)

# Palliative Care Flag: 1 if any ICD code in the list matches these codes
palliative_care_codes = ["Z515"]  # remove icd 9 , "V667" 
diagnosis_counts["palliative_care_flag"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if any(code in palliative_care_codes for code in x) else 0
)

# Cognitive Impairment Flag: 1 if any code is in the specified list (ensure codes are strings)
cognitive_impairment_codes = [
    "G30", "G300", "G301", "G308", "G309",   # Alzheimer's
    "F0150", "F0151", "F01.A0", "F01A1", "F01B0", "F01B1", "F01C0", "F01C1",  # Vascular Dementia
    "F0280", "F0281",                         # Dementia in other diseases
    "F03", "F0390", "F0391",                   # Unspecified Dementia
    "G31", "G3184"                            # Mild Cognitive Impairment
]
diagnosis_counts["cognitive_impairment_flag"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if any(str(code) in cognitive_impairment_codes for code in x) else 0
)

# Delirium Flag: 1 if any code in the list matches these ICD codes
delirium_icd_codes = ["F05", "F051", "F050", "F059"]  # remove icd 9 "2930", "2931" 
diagnosis_counts["delirium"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if isinstance(x, list) and any(code in delirium_icd_codes for code in x) else 0
)

In [97]:
print(diagnosis_counts.head())

   subject_id   hadm_id  num_comorbidities  \
0    10000032  22595853                 11   
1    10000032  22841357                 10   
2    10000032  25742920                 14   
3    10000032  29079034                 16   
4    10000068  25022803                  1   

                                      diagnosis_list primary_diagnosis  \
0  [K766, R188, K740, K7460, K7469, B1920, J449, ...              K766   
1  [B1921, R188, D696, E871, J449, K740, K7460, K...             B1921   
2  [B182, R188, Z9981, K740, K7460, K7469, E875, ...              B182   
3  [I952, I9581, B182, R64, E871, R188, E875, F17...              I952   
4                                            [F1010]             F1010   

   palliative_care_flag  cognitive_impairment_flag  delirium  
0                     0                          0         0  
1                     0                          0         0  
2                     0                          0         0  
3                     0   

In [71]:
# # =============================
# 3. Load and Aggregate ICU Data
# =============================
# Load ICU stays and aggregate: keep the first ICU stay per admission.
icustays = pd.read_csv(
    os.path.join(data_path, "icu/icustays.csv.gz"),
    usecols=["subject_id", "hadm_id", "stay_id", "last_careunit", "los"],
    low_memory=False
)
icustays_agg = icustays.sort_values(["subject_id", "hadm_id", "stay_id"]).groupby(
    ["subject_id", "hadm_id"]
).first().reset_index()

# Count total ICU stays per patient (for history)
icu_history = icustays.groupby("subject_id")["stay_id"].count().reset_index()
icu_history.rename(columns={"stay_id": "prior_icu_admissions"}, inplace=True)

# ---- ICU Chartevents: Process only a subset of items ----
chunksize = 200000
icu_item_ids = [
    # 1. Physiological Parameters
    "220045",  # Heart Rate
    "220052",  # Mean Arterial Blood Pressure
    "220210",  # Respiratory Rate
    "220277",  # Oxygen Saturation (pulseoxymetry)
    "223761",  # Temperature Fahrenheit
    "220739",  # GCS – Eye Opening
    "223900",  # GCS – Verbal Response
    "223901",  # GCS – Motor Response
    "228096",  # Richmond-RAS Scale

    # 2. Laboratory Results
    "220546",  # WBC
    "220228",  # Hemoglobin
    "227457",  # Platelet Count
    "220645",  # Sodium (serum)
    "227442",  # Potassium (serum)
    "220602",  # Chloride (serum)
    "227443",  # HCO3 (serum)
    "225624",  # BUN
    "220615",  # Creatinine (serum)
    "220621",  # Glucose (serum)
    "225668",  # Lactic Acid
    "223830",  # pH (Arterial)
    "220224",  # Arterial O2 pressure (PaO2)
    "220235",  # Arterial CO2 pressure (PaCO2)
    "224828",  # Arterial Base Excess # 3. Intervention-Based Variables
    # 3. Intervention-Based Variables
    "223849",  # Ventilator Mode
    "223836",  # Airway Type
    "223835",  # Inspired O2 Fraction (FiO2)
    "220339",  # PEEP set
    "227210",  # Propofol (Intubation)
    "227211",  # Ketamine (Intubation)
    "227213",  # Vecuronium (Intubation)
    "220120"   # Intra-Aortic Balloon Pump (IABP)
]

chart_file = os.path.join(data_path, "icu/chartevents.csv.gz")
chart_chunks = []
for chunk in pd.read_csv(
    chart_file,
    usecols=["subject_id", "hadm_id", "stay_id", "itemid", "charttime", "valuenum"],
    chunksize=chunksize,
    low_memory=True
):
    # Filter: convert itemid to string and keep only our subset
    chunk = chunk[chunk["itemid"].astype(str).isin(icu_item_ids)]
    chart_chunks.append(chunk)
icu_chartevents = pd.concat(chart_chunks, ignore_index=True)
icu_chartevents.sort_values(["subject_id", "hadm_id", "charttime"], inplace=True)
icu_first = icu_chartevents.groupby(["subject_id", "hadm_id", "itemid"]).first().reset_index()

# Pivot so that each admission is one row with a column per itemid
icu_pivot = icu_first.pivot(
    index=["subject_id", "hadm_id"],
    columns="itemid",
    values="valuenum"
).reset_index()

# (Optional) You may later rename these columns using a mapping if desired.
print("ICU pivot shape:", icu_pivot.shape)
print(icu_pivot.head())

ICU pivot shape: (85242, 31)
itemid  subject_id   hadm_id  220045  220052  220120  220210  220224  220228  \
0         10000032  29079034    91.0     NaN     NaN    24.0     NaN     NaN   
1         10000690  25860671    79.0     NaN     NaN    23.0   123.0     9.5   
2         10000980  26913865    77.0     NaN     NaN    23.0     NaN     NaN   
3         10001217  24597018    86.0     NaN     NaN    18.0     NaN    11.2   
4         10001217  27703517    96.0     NaN     NaN    11.0     NaN    12.3   

itemid  220235  220277  ...  223849  223900  223901  224828  225624  225668  \
0          NaN    98.0  ...     NaN     4.0     6.0     NaN    33.0     NaN   
1         54.0   100.0  ...     NaN     5.0     6.0     0.0    21.0     NaN   
2          NaN   100.0  ...    11.0     5.0     6.0     NaN     NaN     1.7   
3          NaN    99.0  ...     NaN     5.0     6.0     NaN     9.0     NaN   
4          NaN   100.0  ...     NaN     5.0     6.0     NaN    10.0     NaN   

itemid  227442 

In [72]:
id_to_label = {
    220045: "heart_rate",      # Heart Rate
    220052: "map",             # Mean Arterial Blood Pressure
    220210: "resp_rate",       # Respiratory Rate
    220277: "spo2",            # Oxygen Saturation
    223761: "temp",            # Temperature Fahrenheit
    220739: "gcs_eye",         # GCS – Eye Opening
    223900: "gcs_verbal",      # GCS – Verbal Response
    223901: "gcs_motor",       # GCS – Motor Response
    228096: "ras_scale",       # Richmond-RAS Scale
    220546: "wbc",             # White Blood Cell Count
    220228: "hb",              # Hemoglobin
    227457: "plt",             # Platelet Count
    220645: "sodium",          # Sodium (serum)
    227442: "potassium",       # Potassium (serum)
    220602: "chloride",        # Chloride (serum)
    227443: "hco3",            # HCO3 (serum)
    225624: "bun",             # BUN
    220615: "creatinine",      # Creatinine (serum)
    220621: "glucose",         # Glucose (serum)
    225668: "lactate",         # Lactic Acid
    223830: "ph",              # pH (Arterial)
    220224: "pao2",            # Arterial O2 pressure (PaO₂)
    220235: "paco2",           # Arterial CO₂ pressure (PaCO₂)
    224828: "base_excess",     # Arterial Base Excess
    223849: "vent_mode",       # Ventilator Mode
    223836: "airway",          # Airway Type
    223835: "fio2",            # Inspired O₂ Fraction (FiO₂)
    220339: "peep",            # PEEP set
    227210: "propofol",        # Propofol (Intubation)
    227211: "ketamine",        # Ketamine (Intubation)
    227213: "vecuronium",      # Vecuronium (Intubation)
    220120: "iabp"             # Intra-Aortic Balloon Pump (IABP)
}

icu_pivot.rename(columns=lambda col: "icu_" + id_to_label[col]
                                  if col not in ["subject_id", "hadm_id"] and col in id_to_label
                                  else col,
                 inplace=True)



In [73]:
print("ICU pivot shape:", icu_pivot.shape)
print(icu_pivot.head())

ICU pivot shape: (85242, 31)
itemid  subject_id   hadm_id  icu_heart_rate  icu_map  icu_iabp  \
0         10000032  29079034            91.0      NaN       NaN   
1         10000690  25860671            79.0      NaN       NaN   
2         10000980  26913865            77.0      NaN       NaN   
3         10001217  24597018            86.0      NaN       NaN   
4         10001217  27703517            96.0      NaN       NaN   

itemid  icu_resp_rate  icu_pao2  icu_hb  icu_paco2  icu_spo2  ...  \
0                24.0       NaN     NaN        NaN      98.0  ...   
1                23.0     123.0     9.5       54.0     100.0  ...   
2                23.0       NaN     NaN        NaN     100.0  ...   
3                18.0       NaN    11.2        NaN      99.0  ...   
4                11.0       NaN    12.3        NaN     100.0  ...   

itemid  icu_vent_mode  icu_gcs_verbal  icu_gcs_motor  icu_base_excess  \
0                 NaN             4.0            6.0              NaN   
1      

In [ ]:
# # =============================
# 3B. Rename ICU Data colummns 
# =============================

In [75]:
# =============================
# 4. Load and Process Lab Data
# =============================
lab_ids = [
    "51300",  # WBC Count
    "51111",  # Hemoglobin
    "51265",  # Platelet Count
    "52623",  # Sodium
    "52610",  # Potassium
    "52535",  # Chloride
    "51739",  # Total CO2 (as surrogate for bicarbonate)
    "52569",  # Glucose
    "52647",  # BUN
    "52546",  # Creatinine
    "53084",  # ALT
    "53088",  # AST
    "53086",  # Alkaline Phosphatase
    "53089",  # Total Bilirubin
    "50889",  # CRP
    "50862"   # Albumin
]
lab_chunks = []
lab_file = os.path.join(data_path, "hosp/labevents.csv.gz")
for chunk in pd.read_csv(
    lab_file,
    usecols=["subject_id", "hadm_id", "itemid", "charttime", "valuenum"],
    chunksize=200000,
    low_memory=False
):
    # Filter for desired lab tests (cast itemid to string for safe comparison)
    chunk = chunk[chunk["itemid"].astype(str).isin(lab_ids)]
    lab_chunks.append(chunk)
lab_events = pd.concat(lab_chunks, ignore_index=True)
lab_events.sort_values(["subject_id", "hadm_id", "charttime"], inplace=True)
# Keep only the first recorded value for each lab test per admission
lab_first = lab_events.groupby(["subject_id", "hadm_id", "itemid"]).first().reset_index()
lab_pivot = lab_first.pivot(
    index=["subject_id", "hadm_id"],
    columns="itemid",
    values="valuenum"
).reset_index()
# Rename lab columns using a mapping for clarity
lab_pivot.rename(columns={
    50862: "LabH_Alb",
    50889: "LabH_CRP",
    51111: "LabH_Hb",
    51265: "LabH_Plt",
    51300: "LabH_WBC"
}, inplace=True)


In [87]:
print(lab_pivot.head())

itemid  subject_id     hadm_id  LabH_Alb  LabH_CRP  LabH_Hb  LabH_Plt  \
0         10000032  22595853.0       NaN       NaN      NaN      71.0   
1         10000032  22841357.0       2.7       NaN      NaN     137.0   
2         10000032  25742920.0       3.5       NaN      NaN     133.0   
3         10000032  29079034.0       3.8       NaN      NaN      94.0   
4         10000084  23052089.0       4.0       NaN      NaN     263.0   

itemid  LabH_WBC  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  


In [77]:
# =============================
# 5. Load and Aggregate Prescriptions (High-Risk Medications & Drug List)
# =============================
prescriptions = pd.read_csv(
    os.path.join(data_path, "hosp/prescriptions.csv.gz"),
    usecols=["subject_id", "hadm_id", "drug"],
    low_memory=False
)
# Standardize drug names
prescriptions["drug"] = prescriptions["drug"].astype(str).str.lower().str.strip()

# Aggregate all prescribed drugs per admission
all_drugs = prescriptions.groupby(["subject_id", "hadm_id"])["drug"].agg(list).reset_index()

# Define expanded high-risk medications list (based on literature)
high_risk_meds = [
    "lorazepam", "midazolam", "diazepam", "clonazepam", "alprazolam", "temazepam", "chlordiazepoxide",
    "morphine", "fentanyl", "hydromorphone", "oxycodone", "meperidine", "codeine", "tramadol", "buprenorphine",
    "propofol", "dexmedetomidine",
    "haloperidol", "olanzapine", "quetiapine", "risperidone", "ziprasidone", "chlorpromazine", "aripiprazole",
    "diphenhydramine", "hydroxyzine", "promethazine", "oxybutynin", "scopolamine", "benztropine", "atropine",
    "dexamethasone", "prednisone", "methylprednisolone", "hydrocortisone",
    "metoclopramide", "cyclobenzaprine", "baclofen", "amantadine"
]
prescriptions["high_risk_med"] = prescriptions["drug"].apply(
    lambda x: 1 if any(med in x for med in high_risk_meds) else 0
)
# Compute per-admission metrics:
unique_high_risk_med = prescriptions[prescriptions["high_risk_med"] == 1].groupby(
    ["subject_id", "hadm_id"]
)["drug"].nunique().reset_index(name="unique_high_risk_med")

high_risk_med_count = prescriptions.groupby(
    ["subject_id", "hadm_id"]
)["high_risk_med"].sum().reset_index(name="high_risk_med_count")

high_risk_med_flag = prescriptions.groupby(
    ["subject_id", "hadm_id"]
)["high_risk_med"].max().reset_index(name="high_risk_med_flag")

high_risk_med_summary = high_risk_med_flag.merge(
    unique_high_risk_med, on=["subject_id", "hadm_id"], how="left"
).merge(high_risk_med_count, on=["subject_id", "hadm_id"], how="left")
high_risk_med_summary["unique_high_risk_med"] = high_risk_med_summary["unique_high_risk_med"].fillna(0).astype(int)


In [89]:
print(high_risk_med_summary.head())

   subject_id   hadm_id  high_risk_med_flag  unique_high_risk_med  \
0    10000032  22595853                   0                     0   
1    10000032  22841357                   0                     0   
2    10000032  25742920                   1                     2   
3    10000032  29079034                   1                     1   
4    10000084  23052089                   1                     1   

   high_risk_med_count  
0                    0  
1                    0  
2                    3  
3                    2  
4                    2  


In [99]:
# Create an empty DataFrame with the key columns
core_data = pd.DataFrame(columns=["subject_id", "hadm_id"])

In [197]:
# =============================
# 6. Merge All Components into a Final Core Dataset
# =============================

# Start with admissions merged with patients (ensuring one row per admission)
core_data = admissions.merge(patients, on="subject_id", how="inner")

# Merge aggregated diagnosis_counts (one row per admission)
core_data = core_data.merge(
    diagnosis_counts[["subject_id", "hadm_id", "primary_diagnosis","palliative_care_flag", "delirium", "cognitive_impairment_flag", "num_comorbidities"]],
    on=["subject_id", "hadm_id"],
    how="left"
)
# Fill missing flag values with 0 and convert to integer
core_data["palliative_care_flag"] = core_data["palliative_care_flag"].fillna(0).astype(int)
core_data["delirium"] = core_data["delirium"].fillna(0).astype(int)
core_data["cognitive_impairment_flag"] = core_data["cognitive_impairment_flag"].fillna(0).astype(int)

# (Optional) Avoid merging raw diagnoses to prevent duplication
# core_data = core_data.merge(diagnoses, on=["subject_id", "hadm_id"], how="left")

# Merge aggregated ICU stays (first ICU stay per admission)
core_data = core_data.merge(icustays_agg, on=["subject_id", "hadm_id"], how="left")

# Merge ICU history (number of ICU stays per patient)
core_data = core_data.merge(icu_history, on="subject_id", how="left")

# Merge aggregated ICU vitals (icu_pivot)
core_data = core_data.merge(icu_pivot, on=["subject_id", "hadm_id"], how="left")

# Merge high-risk medication summary and the full drug list (all_drugs)
core_data = core_data.merge(high_risk_med_summary, on=["subject_id", "hadm_id"], how="left")
core_data = core_data.merge(all_drugs, on=["subject_id", "hadm_id"], how="left")

# Merge aggregated lab results (lab_pivot)
core_data = core_data.merge(lab_pivot, on=["subject_id", "hadm_id"], how="left")

# Fill any remaining missing numeric fields
core_data["num_comorbidities"] = core_data["num_comorbidities"].fillna(0).astype(int)
core_data["prior_icu_admissions"] = core_data["prior_icu_admissions"].fillna(0).astype(int)


In [199]:
#Linkage Data Quality 
df=core_data
#Count occurrences of each patient (subject_id)
patient_counts = df["subject_id"].value_counts()

# Display summary
print("🔍 Number of Unique Patients:", df["subject_id"].nunique())
print("🔍 Total Rows in Dataset:", df.shape[0])
print("🔍 Distribution of Patient Admissions:")
print(patient_counts.describe())  # Summary statistics for occurrences

🔍 Number of Unique Patients: 223452
🔍 Total Rows in Dataset: 546028
🔍 Distribution of Patient Admissions:
count    223452.000000
mean          2.443603
std           3.574410
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         238.000000
Name: count, dtype: float64


In [201]:
#test any merge issues
print(diagnosis_counts.duplicated(subset=["subject_id", "hadm_id"]).sum())


0


In [203]:
# Print the column names
print(core_data.columns)

# Display the first 5 rows
print(core_data.head())

# Alternatively, get a concise summary including non-null counts and dtypes
print(core_data.info())


Index(['subject_id', 'hadm_id', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'marital_status', 'race',
       'ed_time_spent', 'los_hosp', 'gender', 'anchor_age', 'anchor_year',
       'primary_diagnosis', 'palliative_care_flag', 'delirium',
       'cognitive_impairment_flag', 'num_comorbidities', 'stay_id',
       'last_careunit', 'los', 'prior_icu_admissions', 'icu_heart_rate',
       'icu_map', 'icu_iabp', 'icu_resp_rate', 'icu_pao2', 'icu_hb',
       'icu_paco2', 'icu_spo2', 'icu_peep', 'icu_wbc', 'icu_chloride',
       'icu_creatinine', 'icu_glucose', 'icu_sodium', 'icu_gcs_eye',
       'icu_temp', 'icu_ph', 'icu_fio2', 'icu_airway', 'icu_vent_mode',
       'icu_gcs_verbal', 'icu_gcs_motor', 'icu_base_excess', 'icu_bun',
       'icu_lactate', 'icu_potassium', 'icu_hco3', 'icu_plt', 'icu_ras_scale',
       'high_risk_med_flag', 'unique_high_risk_med', 'high_risk_med_count',
       'drug', 'LabH_Alb', 'LabH_CRP', 'LabH_Hb', 'LabH_Plt', 'LabH_WBC'

In [205]:
# =============================
# 7. Final Summary and Save Output
# =============================
print("  Cleaned dataset prepared. One row per admission.")
print("Unique Patients:", core_data["subject_id"].nunique())
print("Total Rows:", core_data.shape[0])

#    save the final dataset (compressed CSV)
output_path = "D:/MIMIC-IV-Data-Pipeline/processed_data"
os.makedirs(output_path, exist_ok=True)
core_data.to_csv(os.path.join(output_path, "delirium_prediction_data_v8.csv.gz"), index=False, compression="gzip")
print("  Cleaned dataset saved successfully!")

✅ Cleaned dataset prepared. One row per admission.
Unique Patients: 223452
Total Rows: 546028
✅ Cleaned dataset saved successfully!
